In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from scipy.special import expit

2024-08-06 01:25:01.157779: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 01:25:01.165578: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-06 01:25:01.187796: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 01:25:01.222471: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 01:25:01.232843: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-06 01:25:01.260651: I tensorflow/core/platform/cpu_feature_gu

KeyboardInterrupt: 

In [ ]:
# Loads mnist data into variables

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

In [ ]:
# Pixel values by default are [0,255] (where 0 is black and 255 is white)
# We normalize these values for better compatibility with my model
train_images = train_images.astype('float32')/255.0
test_images = test_images.astype('float32')/255.0

In [ ]:
rows = 2
cols = 4

fig, axes = plt.subplots(rows, cols, figsize=(10, 5))

# Commented code shows first 8 images (indices 0-7)

for i in range(rows):
    for j in range(cols):
        index = j + (cols * i)
        axes[i, j].imshow(train_images[index], cmap='gray')
        axes[i, j].axis('off')


# # Show 8 random images in the dataset
# for i in range(rows):
#     for j in range(cols):
#         index = random.randint(0, train_images.shape[0] - 1)
#         axes[i, j].imshow(train_images[index], cmap='gray')
#         axes[i, j].axis('off')

fig.suptitle("MNIST Dataset Sample Images", fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:

# Defines the neuron class
class Neuron:
    def __init__(self, activation, z=0.0):
        self.activation = activation
        self.z = z

    def setActivation(self, num):
        self.activation = num

    def getActivation(self):
        return self.activation

# Gets the max value from an array of neurons
def getMax(l):
    _max = l[0].getActivation()
    for i in range(1, l.size):
        if l[i].getActivation() > _max:
            _max = l[i].getActivation()
    return _max    

# Returns the number (index) which has the highest activation
def getMaxNumber(l):
    max_act = l[0].getActivation()
    max_idx = 0
    for i in range(1, l.size):
        if l[i].getActivation() > max_act:
            max_act = l[i].getActivation()
            max_idx = i
            
    return max_idx

# Takes an image (index) and changes the 1st layer of neuron accordingly
def fillFirstLayer(l, data, index):
    for i in range(28):
        for j in range(28):
            l = np.append(l, Neuron(data[index][i][j]))
            
    return l

# Used to initialize layers 2-4
def fillLayerWithZeroNeurons(l, num):
    for i in range(num):
        l = np.append(l, Neuron(0))

    return l

In [ ]:
# layer 1 will be composed of the pixel values of the image inputted
l1 = np.array([])

# layers 2-3 will be the 'hidden layers' or 'black box' of the network
l2 = np.array([])
l3 = np.array([])

# layer 4 will be the output layer or the head
l4 = np.array([])

# initialize the layers here
l1 = fillFirstLayer(l1, train_images, 0)
l2 = fillLayerWithZeroNeurons(l2, 16)
l3 = fillLayerWithZeroNeurons(l3, 16)
l4 = fillLayerWithZeroNeurons(l4, 10)

In [ ]:
# """
# Here we are assigning random weights and biases
# After this chunk runs all 13,002 parameters have a value
# Weights are between -5 and 5 and biases are between -10 and 10
# """

# l1Weights = np.array([])

# for i in range(l1.size):
#     temp = np.array([])
#     for j in range(l2.size):
#         temp = np.append(temp, np.random.uniform(-5, 5))
#     if l1Weights.size == 0:
#         l1Weights = np.array([temp])
#     else:
#         l1Weights = np.append(l1Weights, [temp], axis=0)    

# print(l1Weights.shape)

# l2Weights = np.array([])

# for i in range(l2.size):
#     temp = np.array([])
#     for j in range(l3.size):
#         temp = np.append(temp, np.random.uniform(-5, 5))
#     if l2Weights.size == 0:
#         l2Weights = np.array([temp])
#     else:
#         l2Weights = np.append(l2Weights, [temp], axis=0)     

# print(l2Weights.shape)

# l3Weights = np.array([])

# for i in range(l3.size):
#     temp = np.array([])
#     for j in range(l4.size):
#         temp = np.append(temp, np.random.uniform(-5, 5))
#     if l3Weights.size == 0:
#         l3Weights = np.array([temp])
#     else:
#         l3Weights = np.append(l3Weights, [temp], axis=0)  

# print(l3Weights.shape)

# l2Biases = np.array([])
# l3Biases = np.array([])
# l4Biases = np.array([])

# for i in range(16):
#     l2Biases = np.append(l2Biases, np.random.uniform(-10, 10))
#     l3Biases = np.append(l3Biases, np.random.uniform(-10, 10))
#     if i < 10:
#         l4Biases = np.append(l4Biases, np.random.uniform(-10, 10))

# print(l1Weights.size + l2Weights.size + l3Weights.size + l2Biases.size + l3Biases.size + l4Biases.size)

In [ ]:
n1Size = 784
n2Size = 16
n3Size = 16
n4Size = 10


l1Weights = np.array([])

for i in range(n1Size):
    temp = np.array([])
    for j in range(n2Size):
        # temp is an array of weights from ith neuron to all 16 neurons in the second layer
        temp = np.append(temp, np.random.normal(loc=0, scale=1))
    if l1Weights.size == 0:
        l1Weights = np.array([temp])
    else:
        l1Weights = np.append(l1Weights, [temp], axis=0)    

print(l1Weights.shape)
print(l1Weights)


l2Weights = np.array([])

for i in range(n2Size):
    temp = np.array([])
    for j in range(n3Size):
        temp = np.append(temp, np.random.normal(loc=0, scale=1))
    if l2Weights.size == 0:
        l2Weights = np.array([temp])
    else:
        l2Weights = np.append(l2Weights, [temp], axis=0)     

print(l2Weights.shape)
print(l2Weights)


l3Weights = np.array([])

for i in range(n3Size):
    temp = np.array([])
    for j in range(n4Size):
        temp = np.append(temp, np.random.normal(loc=0, scale=1))
    if l3Weights.size == 0:
        l3Weights = np.array([temp])
    else:
        l3Weights = np.append(l3Weights, [temp], axis=0)  

        
print(l3Weights.shape)
print(l3Weights)


l2Biases = np.array([])
l3Biases = np.array([])
l4Biases = np.array([])

for i in range(16):
    l2Biases = np.append(l2Biases, np.random.normal(loc=0, scale=1))
    l3Biases = np.append(l3Biases, np.random.normal(loc=0, scale=1))
    if i < 10:
        l4Biases = np.append(l4Biases, np.random.normal(loc=0, scale=1))

print(l2Biases.shape)
print(l3Biases.shape)        
print(l4Biases.shape) 

print(l2Biases)
print(l3Biases)        
print(l4Biases)   

        
        
print(l1Weights.size + l2Weights.size + l3Weights.size + l2Biases.size + l3Biases.size + l4Biases.size)

In [ ]:
# Now we have the network fully set up.
# We have 4 layers of neurons
# 13002 parameters (weights and biases) with the weights being randomized between [-5, 5] and the biases randomized between [-10, 10]
# The 1st layer already has the values of the 1st image and the 2-4th layers all contain neurons initialized at 

In [ ]:
# We are using the sigmoid function as our activation function
def sigmoid(x):

    x = np.float128(x)
    
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_prime(x):
    
    return expit(x) * (1.0 - expit(x))
    

class Network:
    
    def setWeights(self, w1, w2, w3):
        self.w1 = np.copy(w1)
        self.w2 = np.copy(w2)
        self.w3 = np.copy(w3)
        
    def setBiases(self, b2, b3, b4):
        self.b2 = np.copy(b2)
        self.b3 = np.copy(b3)
        self.b4 = np.copy(b4)

    def setNeurons(self, n1, n2, n3, n4):
        self.n1 = np.copy(n1)
        self.n2 = np.copy(n2)
        self.n3 = np.copy(n3)
        self.n4 = np.copy(n4)

    # loads an image into the 1st layer
    def initImage(self, data, index):
        
        self.n1 = np.array([])

        # print(data[index].shape[0])
        # print(data[index].shape[1])

        
        for i in range(data[index].shape[0]):
            for j in range(data[index].shape[1]):
                self.n1 = np.append(self.n1, Neuron(data[index][i][j]))

    def runImage(self):
        
        for i in range(self.n2.size): # Per 2nd layer neuron
            _sum = 0.0
            for j in range(self.n1.size): # Per 1st layer neuron
                _sum += self.n1[j].getActivation() * self.w1[j][i]
            z = _sum + self.b2[i]
            a = expit(z)
            self.n2[i].setActivation(a)
            self.n2[i].z = z

        for i in range(self.n3.size): # Per 3rd layer neuron
            _sum = 0.0
            for j in range(self.n2.size): # Per 2nd layer neuron
                _sum += self.n2[j].getActivation() * self.w2[j][i]
            z = _sum + self.b3[i]
            a = expit(z)
            self.n3[i].setActivation(a)
            self.n3[i].z = z

        for i in range(self.n4.size): # Per 4th layer neuron
            _sum = 0.0
            for j in range(self.n3.size): # Per 3rd layer neuron
                _sum += self.n3[j].getActivation() * self.w3[j][i]
            z = _sum + self.b4[i]
            a = expit(z)
            self.n4[i].setActivation(a)
            self.n4[i].z = z

    def calcLoss(self, target):
        loss = 0.0
        for i in range(self.n4.size):
            if i == target:
                loss += (self.n4[i].getActivation() - 1.0) ** 2.0
            else:
                loss += self.n4[i].getActivation() ** 2.0
        return loss
    
    def isCorrect(self, target):
        return target == getMaxNumber(self.n4)
            
    
    # Backprop
    
    def backProp_n4(self, target):
        y = 0
        
        
        for i in range(self.n3.size):
            for j in range(self.n4.size):
                if (j == target):
                    self.w3_mod[i][j] += self.n3[i].getActivation() * sigmoid_prime(self.n4[j].z) * 2.0 * (self.n4[j].getActivation() - 1.0)
                    
                    
                else:
                    self.w3_mod[i][j] += self.n3[i].getActivation() * sigmoid_prime(self.n4[j].z) * 2.0 * (self.n4[j].getActivation())
            
            
        for i in range(self.n4.size):
            if i == target:
                self.b4_mod[i] += sigmoid_prime(self.n4[i].z) * 2.0 * (self.n4[i].getActivation() - 1.0)
            else:
                self.b4_mod[i] += sigmoid_prime(self.n4[i].z) * 2.0 * (self.n4[i].getActivation())
    
    def backProp_n3(self, target):
        y = 0
        for i in range(self.n2.size):        
            
            for j in range(self.n3.size):
                temp1 = self.n2[i].getActivation() * sigmoid_prime(self.n3[j].z)
                temp2 = 0.0
                for k in range(self.n4.size):
                    if k == target:
                        y = 1
                    else:
                        y = 0
                    temp2 += self.w3[j][k] * sigmoid_prime(self.n4[k].z) * 2.0 * (self.n4[k].getActivation() - y)
                
                self.w2_mod[i][j] += temp1 * temp2





        
        for i in range(self.b3.size):
            
            temp1 = sigmoid_prime(self.n3[i].z)
            temp2 = 0.0
            for j in range(self.b4.size):
                if j == target:
                    y = 1
                else:
                    y = 0
                temp2 += self.w3[i][j] * sigmoid_prime(self.n4[j].z) * 2.0 * (self.n4[j].getActivation() - y)
                
            self.b3_mod[i] += temp1 * temp2
            
        
    
    def backProp_n2(self, target):
        y = 0
        for i in range(self.n1.size):  
            for j in range(self.n2.size):
                temp1 = self.n1[i].getActivation() * sigmoid_prime(self.n2[j].z)


                
                temp2 = 0.0
                for k in range(self.n3.size):
                    temp3 = 0.0
                    for l in range(self.n4.size):
                        if l == target:
                            y = 1
                        else:
                            y = 0
                        temp3 += self.w3[k][l] * sigmoid_prime(self.n4[l].z) * 2.0 * (self.n4[l].getActivation() - y)

                    temp3 *= self.w2[j][k] * sigmoid_prime(self.n3[k].z)
                    temp2 += temp3

                self.w1_mod[i][j] += temp1 * temp2
        
        for i in range(self.b2.size):
            temp1 = sigmoid_prime(self.n2[i].z)

            temp2 = 0.0
            
            for j in range(self.b3.size):

                temp3 = 0.0
                for k in range(self.b4.size):
                    if k == target:
                        y = 1
                    else:
                        y = 0
                    temp3 += self.w3[j][k] * sigmoid_prime(self.n4[k].z) * 2.0 * (self.n4[k].getActivation() - y)

                temp3 *= self.w2[i][j] * sigmoid_prime(self.n3[j].z)
                temp2 += temp3

            self.b2_mod[i] += temp1 * temp2
                    
        
    def backProp(self, target):
        self.backProp_n4(target)
        self.backProp_n3(target)
        self.backProp_n2(target)
    
    def runBatch(self, data, labels, startIdx = 0, batchSize=1000):
        
        self.w3_mod = np.copy(self.w3)
        self.w3_mod.fill(0.0)
        
        self.b4_mod = np.copy(self.b4)
        self.b4_mod.fill(0.0)
        
        self.w2_mod = np.copy(self.w2)
        self.w2_mod.fill(0.0)
        
        self.b3_mod = np.copy(self.b3)
        self.b3_mod.fill(0.0)
        
        
        self.w1_mod = np.copy(self.w1)
        self.w1_mod.fill(0.0)
        
        self.b2_mod = np.copy(self.b2)
        self.b2_mod.fill(0.0)
        
        loss = 0.0
        counter = 0
        for i in range(startIdx, startIdx + batchSize):
            if i % 10 == 0:
                print(i)
                
            self.initImage(data, i)
            self.runImage()
            loss += self.calcLoss(labels[i])
            if self.isCorrect(labels[i]):
                counter += 1
            self.backProp(labels[i])


        # ASSUMING BATCH SIZE == 1000
        loss /= 200.0
        print(loss)
        print(counter / batchSize)

        # ASSUMING BATCH SIZE == 1000
        # ADJUSTED TO MATCH SLOW AHH NETWORK
        
        # May be not needed
        self.w3_mod = np.divide(self.w3_mod, 200.0)
        self.w2_mod = np.divide(self.w2_mod, 200.0)
        self.w1_mod = np.divide(self.w1_mod, 200.0)
        self.b4_mod = np.divide(self.b4_mod, 200.0)
        self.b3_mod = np.divide(self.b3_mod, 200.0)
        self.b2_mod = np.divide(self.b2_mod, 200.0)
        
        for i in range(self.n3.size):
            for j in range(self.n4.size):
                self.w3[i][j] -= self.w3_mod[i][j]
        
        for i in range(self.n2.size):
            for j in range(self.n3.size):
                self.w2[i][j] -= self.w2_mod[i][j]
        
        for i in range(self.n1.size):
            for j in range(self.n2.size):
                self.w1[i][j] -= self.w1_mod[i][j]
        
        for i in range(self.b4.size):
            self.b4[i] -= self.b4_mod[i]
        
        for i in range(self.b3.size):
            self.b3[i] -= self.b3_mod[i]
            
        for i in range(self.b2.size):
            self.b2[i] -= self.b2_mod[i]
        
            
            
        
        
            
            

In [ ]:
firstNetwork = Network()
firstNetwork.setWeights(l1Weights, l2Weights, l3Weights)
firstNetwork.setBiases(l2Biases, l3Biases, l4Biases)
firstNetwork.setNeurons(l1, l2, l3, l4)

In [ ]:
# # Lets do a run with our loaded data

# # Calculates activation of neurons in the second layer
# for i in range(firstNetwork.n2.size): # Per 2nd layer neuron
#     sum = 0
#     for j in range(firstNetwork.n1.size): # Per 1st layer neuron
#         sum += firstNetwork.n1[j].getActivation() * firstNetwork.w1[j][i]
#     z = sum + firstNetwork.b2[i]
#     a = sigmoid(z)
#     firstNetwork.n2[i].setActivation(a)

In [ ]:
# for i in range(16):
#     print(firstNetwork.n2[i].getActivation())

In [ ]:
# # 3rd layer

# for i in range(firstNetwork.n3.size): # Per 3rd layer neuron
#     sum = 0
#     for j in range(firstNetwork.n2.size): # Per 2nd layer neuron
#         sum += firstNetwork.n2[j].getActivation() * firstNetwork.w2[j][i]
#     z = sum + firstNetwork.b3[i]
#     a = sigmoid(z)
#     firstNetwork.n3[i].setActivation(a)

In [ ]:
# for i in range(16):
#     print(firstNetwork.n3[i].getActivation())

In [ ]:
# #head

# for i in range(firstNetwork.n4.size): # Per 4th layer neuron
#     sum = 0
#     for j in range(firstNetwork.n3.size): # Per 3rd layer neuron
#         sum += firstNetwork.n3[j].getActivation() * firstNetwork.w3[j][i]
#     z = sum + firstNetwork.b4[i]
#     a = sigmoid(z)
#     firstNetwork.n4[i].setActivation(a)

In [ ]:
# for i in range(10):
#     print(firstNetwork.n4[i].getActivation())

In [ ]:
# print(train_labels[0])
# print(firstNetwork.calcLoss(5))
# print(getMaxNumber(firstNetwork.n4))
# print(getMax(firstNetwork.n4))

In [ ]:
# del(cost) / del(weight) == a(l-1) * sig'(z) * 2(a - y)
# del(cost) / del(bias) == sig'(z) * 2(a - y)

#### BACKPROP ####

In [ ]:
# firstNetwork.backProp_n4(5)
# print(firstNetwork.w3_mod)
# print(firstNetwork.b4_mod)

# firstNetwork.backProp_n3()
# print(firstNetwork.w2_mod)
# print(firstNetwork.b3_mod)
# firstNetwork.backProp_n2()
# print(firstNetwork.w1_mod)
# print(firstNetwork.b2_mod)

# for i in range(784):
#     for j in range(16):
#         print(firstNetwork.w1_mod[i][j])

In [ ]:
for i in range(0, 7800, 200):
    firstNetwork.runBatch(train_images, train_labels, i)
    
# print("Starting Testing")
# firstNetwork.runBatch(test_images, test_labels) # TODO: Don't backpropagate on test runs
    

In [ ]:
for i in range(1000, 10000, 1000):
    firstNetwork.runBatch(test_images, test_labels, i)

In [ ]:
print(test_labels[9999])
for neuron in firstNetwork.n4:
    print(neuron.getActivation())
    

In [ ]:
for i in range(0, 60000, 1000):
    firstNetwork.runBatch(train_images, train_labels, i)
    
print("Starting Testing")
for i in range(0, 10000, 1000):
    firstNetwork.runBatch(test_images, test_labels, i) 